Shopping Log - Activity Report
https://plum.greenehillfood.coop/civicrm/report/instance/103?reset=1&output=criteria

In [19]:
import os
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import itertools
#from datetimerange import DateTimeRange
import pickle
import sqlalchemy

In [5]:
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping/civi_member_shopping')
#/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports')
os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping/civi_member_shopping') # new ver

## Consider doing QA between several files

In [7]:
shop = pd.read_csv('./memberShopping_20240414.csv')

In [8]:
#rename fields
shop.columns = [i.replace(' ','_') for i in shop.columns]

In [9]:
shop['Activity_Date'] = pd.to_datetime(shop['Activity_Date'], format = "%Y-%m-%d %H:%M")

In [7]:
shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3466 entries, 0 to 3465
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Assignee_Name    0 non-null      float64       
 1   Target_Name      3466 non-null   object        
 2   Source_Email     0 non-null      float64       
 3   Assignee_Email   0 non-null      float64       
 4   Target_Email     3466 non-null   object        
 5   Activity_Type    3466 non-null   object        
 6   Subject          3466 non-null   object        
 7   Activity_Date    3466 non-null   datetime64[ns]
 8   Activity_Status  3466 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 243.8+ KB


Review some characteristics of the dataset

In [9]:
print(shop['Activity_Type'].value_counts())
print(shop['Activity_Status'].value_counts())
print(shop['Subject'].value_counts())
#shop['Activity Type'].value_counts()

Shop    15104
Name: Activity_Type, dtype: int64
Completed    15104
Name: Activity_Status, dtype: int64
Checked in to shop via email      11223
Checked in to shop via barcode     3881
Name: Subject, dtype: int64


In [14]:
#date range 
#QA step <- build out (both of these options work)
type(shop['Activity_Date'].max()) == pd.Timestamp
isinstance(shop['Activity_Date'].max(), pd.Timestamp)

True

Fields of interest: Target_Name, Target_Email, Activity_Type, Subject, Activity_Date

In [58]:
shop['ingest_date'] = datetime.datetime.today()

In [59]:
shop.columns

Index(['Assignee_Name', 'Target_Name', 'Source_Email', 'Assignee_Email',
       'Target_Email', 'Activity_Type', 'Subject', 'Activity_Date',
       'Activity_Status', 'ingest_date'],
      dtype='object')

In [19]:
#QA opportunity: columns and the datatypes match the expected
type(shop.dtypes) # a pandas.Series

pandas.core.series.Series

### Import Current Member roster from CIVI (TODO: decipher how this dataset is used in any workflow)

In [61]:
active_mem = pd.read_csv('./activeDetailedReportCIVI_11012023.csv')

FileNotFoundError: [Errno 2] No such file or directory: './activeDetailedReportCIVI_11012023.csv'

In [4]:
active_mem.columns = [i.replace(' ','_') for i in active_mem.columns]

In [5]:
active_mem['Start_Date'] = pd.to_datetime(active_mem['Start_Date'], format = "%Y-%m-%d")

In [20]:
#establish connection to db and export the 'shop_log' table to db
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database)

engine = sqlalchemy.create_engine(url)
#, connect_args={'options': '-csearch_path=schema_name'}, isolation_level="AUTOCOMMIT"

In [21]:
#is there a difference between current DataFrame and pre-existing shop_log table?
from sqlalchemy import inspect
insp = inspect(engine)
#returns table metadata in a list of dictionaries
metatable = insp.get_columns('shop_log')


In [30]:

name,type_,*_ = metatable[0].items()
    #print(name)
print(type_)
    #print(f'Name: {name['name']}| Type:{type_['type']}')

('type', TEXT())


In [54]:
def assign(i):
    name,type_,*_ = i
    return *name,type_ #unpack the tuple at runtime; the return statement = ea element in the resulting list (the list comprehension)
[assign(list(i.items())) for i in metatable]
#[i.items() for i in metatable]

[('name', 'Target_Name', ('type', TEXT())),
 ('name', 'Target_Email', ('type', TEXT())),
 ('name', 'Activity_Type', ('type', TEXT())),
 ('name', 'Subject', ('type', TEXT())),
 ('name', 'Activity_Date', ('type', DATETIME())),
 ('name', 'ingest_date', ('type', DATETIME()))]

In [60]:
#load the new table
with engine.connect() as conn:
    shop.loc[:,['Target_Name', 'Target_Email', 'Activity_Type', 'Subject', 'Activity_Date','ingest_date']].to_sql('shop_log', con=engine, if_exists='replace', index=False)